In [9]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [4]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [10]:
# Load Environment Variables
load_dotenv(override=True)

# Gemini Environment Variables
gemini_api_key = env_to_str('GEMINI_API_KEY')
gemini_base_url = env_to_str('GEMINI_BASE_URL')
model_gemini_flash = env_to_str('MODEL_GEMINI_FLASH')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [11]:
# Agent Workflow Instructions

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [12]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="gemini_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: gemini_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [13]:
# Gemini Model
gemini_model = OpenAIChatCompletionsModel(
    model=model_gemini_flash,
    openai_client=AsyncOpenAI(
        api_key=gemini_api_key, 
        base_url=gemini_base_url
    )
)

In [14]:
# Agents

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=gemini_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=gemini_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=gemini_model
)

In [16]:
sales_agent1.name

'Professional Sales Agent'

In [ ]:
# Phoenix Tracer
async def phoenix_tracer(trace_name, message, *agent):
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span(trace_name):
        
        # Add custom attributes to the trace
        current_span = otel_trace.get_current_span()
        current_span.set_attribute("user.request", message)
        current_span.set_attribute("agent.name", agent.name)
        current_span.set_attribute("model.name", model_gemini_flash)
        
        result = await Runner.run(agent, message)
        
        # Add response attributes
        current_span.set_attribute("response.length", len(result.final_output))
        current_span.set_attribute("response.preview", result.final_output[:100])
        
        display(Markdown(result.final_output))
        return result.final_output
 

In [ ]:
result = await phoenix_tracer()